## Amazon Bedrock Agentcore Browser tool tutorial 


This tutorial demonstrates how to use AgentCore Bedrock Browser to:
1. Create a BrowserClient with a session
2. Use Browser Use agent to drive the browser actions using natural language
3. See results via logging


## Prerequisites
- AWS account with Bedrock AgentCore Browser tool access
- You have the necessary IAM permissions to create and manage browser tool resources
- Required Python packages installed(including boto3 & bedrock-agentcore)
- browser-use sdk instaled


## Your IAM execution role should have the following IAM policy attached

~~~ {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:CreateBrowser",
                "bedrock-agentcore:StartBrowserSession",
                "bedrock-agentcore:InvokeBrowser",
                "bedrock-agentcore:StopBrowserSession",
                "bedrock-agentcore:ListBrowsers",
                "bedrock-agentcore:GetBrowser"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/browser*"
        }
    ]
}

## How it works

A browser tool sandbox is a secure execution environment that enables AI agents to safely interact with web browsers. When a user makes a request, the Large Language Model (LLM) selects appropriate tools and translates commands. These commands are executed within a controlled sandbox environment containing a headless browser and hosted library server (using tools like Playwright). The sandbox provides isolation and security by containing web interactions within a restricted space, preventing unauthorized system access. The agent receives feedback through screenshots and can perform automated tasks while maintaining system security. 

![architecture local](../images//browser-tool.png)

## 1. Setting Up the Environment

First, let's install and import the necessary libraries to initiaize the browser tool sandbox client. 

In [ ]:
%pip install "browser-use<0.3.3" ## Need this version to over come this bug: 
%pip install langchain_aws

In [ ]:
from bedrock_agentcore.tools.browser_client import BrowserClient
from browser_use import Agent
from browser_use.browser.session import BrowserSession
from browser_use.browser import BrowserProfile
from langchain_aws import ChatBedrockConverse
from rich.console import Console
from contextlib import suppress
import time 
import boto3
import asyncio

In [ ]:
console = Console()

## 2. Setup the browser client
We will setup the browser client and wait for it to be ready. Then, we will generate web-socket url and headers 
 


In [ ]:
region = "us-west-2" ## Replace with your region 
client = BrowserClient(region)
client.start()

#Wait for browser to be ready 
for i in range(10, 0, -1):
    print(f"\r   {i} seconds remaining...", end="", flush=True)
    time.sleep(1)
console.print("\r   Browser ready! " )

# Extract ws_url and headers
ws_url, headers = client.generate_ws_headers()

## 4. Helper function to run browser task
Run a browser automation task using browser-use Agent 




In [ ]:
async def run_browser_task(browser_session: BrowserSession, bedrock_chat: ChatBedrockConverse, task: str) -> None:
    """
    Run a browser automation task using browser_use
    
    Args:
        browser_session: Existing browser session to reuse
        bedrock_chat: Bedrock chat model instance
        task: Natural language task for the agent
    """
    try:
        # Show task execution
        console.print(f"\n[bold blue]🤖 Executing task:[/bold blue] {task}")
        
        # Create and run the agent
        agent = Agent(
            task=task,
            llm=bedrock_chat,
            browser_session=browser_session
        )
        
        # Run with progress indicator
        with console.status("[bold green]Running browser automation...[/bold green]", spinner="dots"):
            await agent.run()
        
        console.print("[bold green]✅ Task completed successfully![/bold green]")
        
    except Exception as e:
        console.print(f"[bold red]❌ Error during task execution:[/bold red] {str(e)}")
        import traceback
        if console.is_terminal:
            traceback.print_exc()

## 5. Invoke the browser task using Browser-use profile
Create a persistent browser session using CDP WebSocket connection and initialize Bedrock Claude model for automated web tasks. Handle session lifecycle with proper cleanup and execute browser automation tasks via AI-driven commands.

In [ ]:
# Create persistent browser session and model
browser_session = None
bedrock_chat = None

try:
    # Create browser profile with headers
    browser_profile = BrowserProfile(
        headers=headers,
        timeout=1500000,  # 150 seconds timeout
    )
    
    # Create a browser session with CDP URL and keep_alive=True for persistence
    browser_session = BrowserSession(
        cdp_url=ws_url,
        browser_profile=browser_profile,
        keep_alive=True  # Keep browser alive between tasks
    )
    
    # Initialize the browser session
    console.print("[cyan]🔄 Initializing browser session...[/cyan]")
    await browser_session.start()
    
    # Create ChatBedrockConverse once
    bedrock_chat = ChatBedrockConverse(
        model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
        region_name="us-west-2"
    )
    
    console.print("[green]✅ Browser session initialized and ready for tasks[/green]\n")

    task = "Search for a coffee maker on amazon.com and extract details of the first one"

    await run_browser_task(browser_session, bedrock_chat, task)

finally:
    # Close the browser session
    if browser_session:
        console.print("\n[yellow]🔌 Closing browser session...[/yellow]")
        with suppress(Exception):
            await browser_session.close()
        console.print("[green]✅ Browser session closed[/green]")


## 6. Cleanup
Stop the browser session if it hasn't been

In [ ]:
client.stop()
print("Browser session stopped successfully!")